In [1]:
from __future__ import print_function

import tensorflow as tf
from tensorflow.contrib import rnn

from tensorflow.examples.tutorials.mnist import input_data
mnist=input_data.read_data_sets("Mnist",one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting Mnist\train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting Mnist\train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting Mnist\t10k-images-idx3-ubyte.gz
Extracting Mnist\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [9]:
learing_rate=0.001
training_steps=10000
batch_size=128
display_step=200

#n_input=784
n_classes=10
n_input=28
timesteps=28
n_hidden=128

X=tf.placeholder("float",[None,timesteps,n_input])
Y=tf.placeholder("float",[None,n_classes])

In [10]:
weights={
    'out':tf.Variable(tf.random_normal([n_hidden,n_classes]))
}
biases={
    'out':tf.Variable(tf.random_normal([n_classes]))
}

In [16]:
def RNN(x,weights,biases):
    #prepare data shape to match 'rnn' function requirements
    #current data input shape:(batch_size,timesteps,n_input)
    #required shape: 'timesteps' tensors list of shape(batch_size, n_input)
    
    #unstack to get a list of 'timesteps' tensors of shape(batch_size,n_input)
    x=tf.unstack(x,timesteps,1)
    #     Unpacks the given dimension of a rank-R tensor into rank-(R-1) tensors.

    #     Unpacks num tensors from value by chipping it along the axis dimension.
    #     If num is not specified (the default), it is inferred from value's shape.

    #define a lstm cell with tensorflow
   
    lstm_cell=rnn.BasicLSTMCell(n_hidden,forget_bias=1.0)
    outputs,states=rnn.static_rnn(lstm_cell,x,dtype=tf.float32)
    return tf.matmul(outputs[-1],weights['out'])+biases['out']

In [21]:
#  with tf.variable_scope('train',reuse=True):
logits=RNN(X,weights,biases)
with tf.variable_scope('train'):
    prediction =tf.nn.softmax(logits)
loss_op=tf.reduce_mean(tf.softmax_cross_entropy_with_logits(
    logits=logits,labels=Y))
train_op=tf.train.GRadientDescentOptimizer(learining_rate).minimize(loss_op)

#evaluate model(with test logits, for dropout to be disabled)
correct_pred=tf.equal(tf.argmax(prediction,1),tf.argmax(Y,1))
accuracy=tf.reduce_mean(tf.cast(correct_pred,tf.float32))

init=tf.global_variables_initializer()

ValueError: Variable rnn/basic_lstm_cell/kernel already exists, disallowed. Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope? Originally defined at:

  File "E:\anacondaSF\envs\tensorflow\lib\site-packages\tensorflow\python\framework\ops.py", line 1717, in __init__
    self._traceback = tf_stack.extract_stack()
  File "E:\anacondaSF\envs\tensorflow\lib\site-packages\tensorflow\python\framework\ops.py", line 3155, in create_op
    op_def=op_def)
  File "E:\anacondaSF\envs\tensorflow\lib\site-packages\tensorflow\python\util\deprecation.py", line 454, in new_func
    return func(*args, **kwargs)


In [ ]:
# Start training
with tf.Session() as sess:

    # Run the initializer
    sess.run(init)

    for step in range(1, training_steps+1):
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        # Reshape data to get 28 seq of 28 elements
        batch_x = batch_x.reshape((batch_size, timesteps, num_input))
        # Run optimization op (backprop)
        sess.run(train_op, feed_dict={X: batch_x, Y: batch_y})
        if step % display_step == 0 or step == 1:
            # Calculate batch loss and accuracy
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: batch_x,
                                                                 Y: batch_y})
            print("Step " + str(step) + ", Minibatch Loss= " + \
                  "{:.4f}".format(loss) + ", Training Accuracy= " + \
                  "{:.3f}".format(acc))

    print("Optimization Finished!")

    # Calculate accuracy for 128 mnist test images
    test_len = 128
    test_data = mnist.test.images[:test_len].reshape((-1, timesteps, num_input))
    test_label = mnist.test.labels[:test_len]
    print("Testing Accuracy:", \
        sess.run(accuracy, feed_dict={X: test_data, Y: test_label}))